In [1]:
import numpy as np 
import pandas as pd 
import re
import os
import glob
import pickle
from sklearn.model_selection import train_test_split 
import librosa

In [2]:
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import concatenate
from keras import optimizers
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [3]:
INPUT_FOLDER = "input/"
# INPUT_FOLDER = "../input/"
print(os.listdir(INPUT_FOLDER))

['train_curated.csv', 'train_noisy.csv', 'sample_submission.csv', 'train_noisy', 'train_curated', 'test']


In [4]:
TRAIN_CURATED_PATH = INPUT_FOLDER + "train_curated.csv"
TRAIN_NOISY_PATH = INPUT_FOLDER + "train_noisy.csv"
SAMPLE_SUBMISSION_PATH = INPUT_FOLDER + "sample_submission.csv"
TRAIN_CURATED = INPUT_FOLDER + "train_curated/"
TRAIN_NOISY = INPUT_FOLDER + "train_noisy/"
TEST = INPUT_FOLDER + "test/"

train_curated = pd.read_csv(TRAIN_CURATED_PATH)
train_noisy = pd.read_csv(TRAIN_NOISY_PATH)
sample = pd.read_csv(SAMPLE_SUBMISSION_PATH)

In [5]:
def one_hot(labels, src_dict):
    ar = np.zeros([len(labels), len(src_dict)])
    for i, label in enumerate(labels):
        label_list = label.split(',')
        for la in label_list:
            ar[i, src_dict[la]] = 1
    return ar

In [6]:
target_names = sample.columns[1:]
num_targets = len(target_names)

src_dict = {target_names[i]:i for i in range(num_targets)}
src_dict_inv = {i:target_names[i] for i in range(num_targets)}

In [7]:
num_freq = 128
len_div = 256

In [9]:
file_name = train_curated['fname'].values

div = 500
num_batch = len(file_name) // div
rest = len(file_name) % div

pos = [range(div*k, div*(k+1)) for k in range(num_batch)]
pos.append(range(div*num_batch, div*num_batch+rest))

In [11]:

y_proc_tmp = one_hot(train_curated['labels'], src_dict)

for k in range(num_batch+1):
    X_proc_ = np.zeros([1, num_freq, len_div])
    y_proc_ = np.zeros([1,80])
    for i, file in enumerate(file_name[pos[k]]):
        wavfile = TRAIN_CURATED + file
        y_proc, sr = librosa.load(wavfile)
        S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
        log_S = librosa.power_to_db(S, ref=np.max)
        X_proc = (log_S + 80) / 40 - 1

        num_div = X_proc.shape[1] // len_div
        num_pad = len_div - X_proc.shape[1] % len_div
        redidual_amp = np.zeros([num_freq, num_pad])
        dum = np.hstack([X_proc, redidual_amp])
        X_proc_ = np.vstack([X_proc_, np.array(np.split(dum, num_div+1,1))])
        for _ in range(num_div+1):
            y_proc_ = np.vstack([y_proc_, y_proc_tmp[i+div*k]])

    X = X_proc_[1:]
    y = y_proc_[1:]
    X = X.reshape([-1, num_freq, len_div, 1])
    
    print('iter No.{} is done.'.format(k))

    with open('out/train_arr_{}.pickle'.format(k), 'wb') as f:
        pickle.dump(X, f)
        pickle.dump(y, f)

iter No.0 is done.
iter No.1 is done.
iter No.2 is done.
iter No.3 is done.
iter No.4 is done.
iter No.5 is done.
iter No.6 is done.
iter No.7 is done.
iter No.8 is done.
iter No.9 is done.


In [12]:
X_proc_ = np.zeros([1, num_freq, len_div, 1])
y_proc_ = np.zeros([1,80])

for k in range(num_batch+1):
    with open('out/train_arr_{}.pickle'.format(k), 'rb') as f:
        X_part = pickle.load(f)
        y_part = pickle.load(f)

    X_proc_ = np.vstack([X_proc_, X_part])
    y_proc_ = np.vstack([y_proc_, y_part])
    
X = X_proc_[1:]
y = y_proc_[1:]
print(X.shape)
print(y.shape)
with open('train_arr.pickle', 'wb') as f:
    pickle.dump(X, f)
    pickle.dump(y, f)

(9377, 128, 256, 1)
(9377, 80)


In [9]:
with open('train_arr.pickle', 'rb') as f:
    X = pickle.load(f)
    y = pickle.load(f)

In [8]:
file_name = train_noisy['fname'].values

div = 500
num_batch = len(file_name) // div
rest = len(file_name) % div

pos = [range(div*k, div*(k+1)) for k in range(num_batch)]
pos.append(range(div*num_batch, div*num_batch+rest))


y_proc_tmp = one_hot(train_noisy['labels'], src_dict)

for k in range(num_batch+1):
    X_proc_ = np.zeros([1, num_freq, len_div])
    y_proc_ = np.zeros([1,80])
    for i, file in enumerate(file_name[pos[k]]):
        wavfile = TRAIN_NOISY + file
        y_proc, sr = librosa.load(wavfile)
        S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
        log_S = librosa.power_to_db(S, ref=np.max)
        X_proc = (log_S + 80) / 40 - 1

        num_div = X_proc.shape[1] // len_div
        num_pad = len_div - X_proc.shape[1] % len_div
        redidual_amp = np.zeros([num_freq, num_pad])
        dum = np.hstack([X_proc, redidual_amp])
        X_proc_ = np.vstack([X_proc_, np.array(np.split(dum, num_div+1,1))])
        for _ in range(num_div+1):
            y_proc_ = np.vstack([y_proc_, y_proc_tmp[i+div*k]])

    X = X_proc_[1:]
    y = y_proc_[1:]
    X = X.reshape([-1, num_freq, len_div, 1])
    
    print('iter No.{} is done.'.format(k))

    with open('out/train_noisy_arr_{}.pickle'.format(k), 'wb') as f:
        pickle.dump(X, f)
        pickle.dump(y, f)
        
X_proc_ = np.zeros([1, num_freq, len_div, 1])
y_proc_ = np.zeros([1,80])


iter No.0 is done.
iter No.1 is done.
iter No.2 is done.
iter No.3 is done.
iter No.4 is done.
iter No.5 is done.
iter No.6 is done.
iter No.7 is done.
iter No.8 is done.
iter No.9 is done.
iter No.10 is done.
iter No.11 is done.
iter No.12 is done.
iter No.13 is done.
iter No.14 is done.
iter No.15 is done.
iter No.16 is done.
iter No.17 is done.
iter No.18 is done.
iter No.19 is done.
iter No.20 is done.
iter No.21 is done.
iter No.22 is done.
iter No.23 is done.
iter No.24 is done.
iter No.25 is done.
iter No.26 is done.
iter No.27 is done.
iter No.28 is done.
iter No.29 is done.
iter No.30 is done.
iter No.31 is done.
iter No.32 is done.
iter No.33 is done.
iter No.34 is done.
iter No.35 is done.
iter No.36 is done.
iter No.37 is done.
iter No.38 is done.
iter No.39 is done.


In [ ]:
for k in range(num_batch+1):
    with open('out/train_noisy_arr_{}.pickle'.format(k), 'rb') as f:
        X_part = pickle.load(f)
        y_part = pickle.load(f)

    X_proc_ = np.vstack([X_proc_, X_part])
    y_proc_ = np.vstack([y_proc_, y_part])

In [ ]:
X = X_proc_[1:]
y = y_proc_[1:]
print(X.shape)
print(y.shape)
with open('train_noisy_arr.pickle', 'wb') as f:
    pickle.dump(X, f)
    pickle.dump(y, f)

In [ ]:
with open('train_noisy_arr.pickle', 'rb') as f:
    X_noisy = pickle.load(f)
    y_noisy = pickle.load(f)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [9]:
inputs = Input(shape=(num_freq,len_div,1), name='input')

dense_list = []

## Block 1
conv1 = Conv2D(4, (19, 19),activation='relu',padding='same',name='conv1')(inputs)
pool1 = MaxPooling2D((19, 19),strides=(1, 1),padding='same',name='pool1')(conv1)
norm1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1')(pool1)
drop1 = Dropout(rate=0.05)(norm1)

conv1_1 = Conv2D(4, (11, 11),activation='relu',padding='same',name='conv1_1')(drop1)
pool1_1 = MaxPooling2D((5, 5),strides=(5, 5),padding='same',name='pool1_1')(conv1_1)
norm1_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1_1')(pool1_1)
drop1_1 = Dropout(rate=0.05)(norm1_1)

flatten1 = Flatten(name='flatten1')(drop1)
dense1 = Dense(16, name='dense1')(flatten1)
act1 = Activation('relu',name='act1')(dense1)
dense_list.append(act1)

## Block 2
conv2 = Conv2D(4, (13, 13),activation='relu',padding='same',name='conv2')(inputs)
pool2 = MaxPooling2D((13, 13), strides=(1, 1), padding='same',name='pool2')(conv2)
norm2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2')(pool2)
drop2 = Dropout(rate=0.05)(norm2)

conv2_1 = Conv2D(4, (7, 7),activation='relu',padding='same',name='conv2_1')(drop2)
pool2_1 = MaxPooling2D((7, 7), strides=(5, 5), padding='same',name='pool2_1')(conv2_1)
norm2_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2_1')(pool2_1)
drop2_1 = Dropout(rate=0.05)(norm2_1)

flatten2 = Flatten(name='flatten2')(drop2_1)
dense2 = Dense(16, name='dense2')(flatten2)
act2 = Activation('relu',name='act2')(dense2)
dense_list.append(act2)

## Block 3
conv3 = Conv2D(8, (11, 11), activation='relu',padding='same',name='conv3')(inputs)
pool3 = MaxPooling2D((11, 11), strides=(2, 2), padding='same',name='pool3')(conv3)
norm3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm3')(pool3)
drop3 = Dropout(rate=0.05)(norm3)

conv3_1 = Conv2D(8, (5, 5), activation='relu',padding='same',name='conv3_1')(drop3)
pool3_1 = MaxPooling2D((5, 5), strides=(2, 2), padding='same',name='pool3_1')(conv3_1)
norm3_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm3_1')(pool3_1)
drop3_1 = Dropout(rate=0.05)(norm3_1)

flatten3 = Flatten(name='flatten3')(drop3_1)
dense3 = Dense(16, name='dense3')(flatten3)
act3 = Activation('relu',name='act3')(dense3)
dense_list.append(act3)

## Block 4
conv4 = Conv2D(8, (9, 9),activation='relu',padding='same',name='conv4')(inputs)
pool4 = MaxPooling2D((9, 9), strides=(2, 2), padding='same',name='pool4')(conv4)
norm4 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm4')(pool4)
drop4 = Dropout(rate=0.05)(norm4)

conv4_1 = Conv2D(8, (3, 3),activation='relu',padding='same',name='conv4_1')(drop4)
pool4_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='same',name='pool4_1')(conv4_1)
norm4_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm4_1')(pool4_1)
drop4_1 = Dropout(rate=0.05)(norm4_1)

flatten4 = Flatten(name='flatten4')(drop4_1)
dense4 = Dense(16, name='dense4')(flatten4)
act4 = Activation('relu',name='act4')(dense4)
dense_list.append(act4)

concat = concatenate(dense_list, name='concat', axis=1)

dense2 = Dense(80, name='dense_all')(concat)
pred = Activation('softmax',name='pred')(dense2)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model = Model(inputs=inputs, outputs=pred)

In [10]:
def tf_one_sample_positive_class_precisions(y_true, y_pred) :
    num_samples, num_classes = y_pred.shape
    
    # find true labels
    pos_class_indices = tf.where(y_true > 0) 
    
    # put rank on each element
    class_rankings = tf.nn.top_k(y_pred, k=num_classes).indices 
    
    #pick_up ranks
    num_correct_until_correct = tf.gather_nd(class_rankings, pos_class_indices) 
    
    # add one for division for "presicion_at_hits"
    num_correct_until_correct_one = tf.add(num_correct_until_correct, 1) 
    num_correct_until_correct_one = tf.cast(num_correct_until_correct_one, tf.float32)
    
    # generate tensor [num_sample, predict_rank], 
    # top-N predicted elements have flag, N is the number of positive for each sample.
    sample_label = pos_class_indices[:, 0]   
    sample_label = tf.reshape(sample_label, (-1, 1))
    sample_label = tf.cast(sample_label, tf.int32)
    num_correct_until_correct = tf.reshape(num_correct_until_correct, (-1, 1))  
    retrieved_class_true_position = tf.concat((sample_label, 
                                               num_correct_until_correct), axis=1)
    retrieved_pos = tf.ones(shape=tf.shape(retrieved_class_true_position)[0], dtype=tf.int32)
    retrieved_class_true = tf.scatter_nd(retrieved_class_true_position, 
                                         retrieved_pos, 
                                         tf.shape(y_pred))

    # cumulate predict_rank
    retrieved_cumulative_hits = tf.cumsum(retrieved_class_true, axis=1)

    # find positive position
    pos_ret_indices = tf.where(retrieved_class_true > 0)

    # find cumulative hits
    correct_rank = tf.gather_nd(retrieved_cumulative_hits, pos_ret_indices)  
    correct_rank = tf.cast(correct_rank, tf.float32)

    # compute presicion
    precision_at_hits = tf.truediv(correct_rank, num_correct_until_correct_one)
 
    return pos_class_indices, precision_at_hits

def tf_lwlrap(y_true, y_pred):
    num_samples, num_classes = y_pred.shape
    
    pos_class_indices, precision_at_hits = (tf_one_sample_positive_class_precisions(y_true, y_pred))
    
    pos_flgs = tf.cast(y_true > 0, tf.int32)
    
    labels_per_class = tf.reduce_sum(pos_flgs, axis=0)
    
    weight_per_class = tf.truediv(tf.cast(labels_per_class, tf.float32),
                                  tf.cast(tf.reduce_sum(labels_per_class), tf.float32))
    
    sum_precisions_by_classes = tf.zeros(shape=(num_classes), dtype=tf.float32)  
    
    class_label = pos_class_indices[:,1]

    sum_precisions_by_classes = tf.unsorted_segment_sum(precision_at_hits,
                                                        class_label,
                                                       num_classes)
    
    labels_per_class = tf.cast(labels_per_class, tf.float32)
    labels_per_class = tf.add(labels_per_class, 1e-7)
    per_class_lwlrap = tf.truediv(sum_precisions_by_classes,
                                  tf.cast(labels_per_class, tf.float32))
    
    out = tf.cast(tf.tensordot(per_class_lwlrap, weight_per_class, axes=1), dtype=tf.float32)
    return out

In [17]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
# model.summary()

In [19]:
datagen = ImageDataGenerator(
           rotation_range=0,
           width_shift_range=16,
           height_shift_range=0,
           shear_range=0,
           zoom_range=0,
           horizontal_flip=False,
           vertical_flip=False)

In [ ]:
with open('train_arr.pickle', 'rb') as f:
    X_train = pickle.load(f)
    y_train = pickle.load(f)

datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=30)

Epoch 1/30
294/293 [==============================] - 114s 387ms/step - loss: 3.7492 - acc: 0.2166
Epoch 2/30
294/293 [==============================] - 111s 378ms/step - loss: 3.1205 - acc: 0.3089
Epoch 3/30
294/293 [==============================] - 111s 377ms/step - loss: 2.9840 - acc: 0.3394
Epoch 4/30
294/293 [==============================] - 112s 382ms/step - loss: 2.8626 - acc: 0.3560
Epoch 5/30
294/293 [==============================] - 113s 385ms/step - loss: 2.7812 - acc: 0.3793
Epoch 6/30
210/293 [====================>.........] - ETA: 31s - loss: 2.7522 - acc: 0.3738

In [ ]:
for k in range(num_batch+1):
    with open('out/train_noisy_arr_{}.pickle'.format(k), 'rb') as f:
        X_train = pickle.load(f)
        y_train = pickle.load(f)

    datagen.fit(X_train)
    model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=len(X_train) / 32, epochs=5)

In [ ]:
with open('20190505model_curated_noisy.pickle', 'wb') as f:
    pickle.dump(model, f)

In [9]:
div = 300
num_batch = len(file_name) // div
rest = len(file_name) % div

pos = [range(div*k, div*(k+1)) for k in range(num_batch)]
pos.append(range(div*num_batch, div*num_batch+rest))

In [10]:
pos

[range(0, 300), range(300, 600), range(600, 900), range(900, 1120)]

In [26]:
for k in range(num_batch+1):
    X_proc_ = []
    for file in file_name[pos[k]]:
        wavfile = file
        y_proc, sr = librosa.load(wavfile)
        S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
        log_S = librosa.power_to_db(S, ref=np.max)
        X_proc = (log_S + 80) / 40 - 1

        num_div = X_proc.shape[1] // len_div
        num_pad = len_div - X_proc.shape[1] % len_div
        redidual_amp = np.zeros([num_freq, num_pad])
        dum = np.hstack([X_proc, redidual_amp])
        X_proc_.append(np.array(np.split(dum, num_div+1,1)))
    
    print('iter No.{} is done.'.format(k))

    with open('out/test_arr_{}.pickle'.format(k), 'wb') as f:
        pickle.dump(X_proc_, f)

iter No.0 is done.
iter No.1 is done.
iter No.2 is done.
iter No.3 is done.


In [27]:
X_proc_ = []

for k in range(num_batch+1):
    with open('out/test_arr_{}.pickle'.format(k), 'rb') as f:
        X_part = pickle.load(f)
    X_proc_.extend(X_part)
    
with open('test_arr.pickle', 'wb') as f:
    pickle.dump(X_proc_, f)
    pickle.dump(file_name, f)

In [29]:
len(X_proc_)

1120

In [ ]:
X_test_list = []

for file in filename:
    wavfile = file
    y_proc, sr = librosa.load(wavfile)
    S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
    log_S = librosa.power_to_db(S, ref=np.max)
    X_proc = (log_S + 80) / 40 - 1
    
    num_div = X_proc.shape[1] // len_div
    num_pad = len_div - X_proc.shape[1] % len_div
    redidual_amp = np.zeros([num_freq, num_pad])
    dum = np.hstack([X_proc, redidual_amp])
    X_test_list.append(np.array(np.split(dum, num_div+1,1)))

In [ ]:
pred_list = []
for X_test in X_test_list:
    pred = model.predict(X_test.reshape([-1, num_freq, len_div,1])).sum(axis=0) / len(X_test)
    pred_list.append(pred)
y_pred = np.array(pred_list)

In [ ]:
sound_names = sample.columns[1:]

In [ ]:
names = []
for f in filename:
    names.append(f.split('/')[-1])

In [ ]:
se_file = pd.Series(names, name='fname')
label = pd.DataFrame(y_pred, columns=sound_names)

In [ ]:
sub_df = pd.concat([se_file, label], axis=1)

In [ ]:
sub_df.to_csv('submission.csv', index=False)